<a href="https://colab.research.google.com/github/lithium0003/DeepLabCut/blob/master/dlc_tpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Before Colabratory process
Colabratory system has no GUI, you need to run in local system label_frames step.

```  python
import deeplabcut
deeplabcut.crate_new_project('colab','test','/home/user/video/',videotype='.mp4')
config_path = '/home/user/deeplabcut/colab-test-2019-08-16/config.yaml'
deeplabcut.extract_frames(config_path, mode='automatic', userfeedback=False)
deeplabcut.label_frames(config_path)
```

After labeling, copy the project folder to Google Drive.


# Connect to Google Drive
To access your data, connect to your Google Drive.

In [0]:
from google.colab import drive
drive.mount('/content/drive')

# Connect to Google Cloud Storage
To access Storage bucket, allow access for tensorflow api.


If you have a bucket named 'test_20190814' and access is allowed successfully,
run this command and no errors are shown.
```
!gsutil ls gs://test_20190814
```



In [0]:
from google.colab import auth
auth.authenticate_user()
!gsutil ls gs://test_20190814

# Install DeepLabCut for TPU
Install DeepLabCut for TPU.

In [0]:
!pip install git+https://github.com/lithium0003/DeepLabCut.git

# Set environment values
In colabratory, there are no GUI system so some functions are disabled.

For TPU system, new environment value is needed.

In [0]:
import os
os.environ["DLClight"]="True"
os.environ["Colab"]="True"
os.environ["TPU"]="True"

# Import DeepLabCut
Some messege are shown for warning no GUI system mode.

In [0]:
import deeplabcut

# Setup config_path
If you stored deeplabcut project folder 'some_dir' in Google Drive,
config_path is begined by '/content/drive/My Drive/'.

In [0]:
config_path='/content/drive/My Drive/some_dir/colab-test-2019-08-15/config.yaml'
config_path

# Make trainig dataset
TPU system is needed to put data in Google Cloud Storage.
You need to set 'train_base_path' in config.yaml to somewhere gs://bucketname.
And also, 'project_path' in config.yaml is need to fix, somewhere path started by '/content/drive/My Drive/'.

After modification of config.yaml, run crate_training_dataset.

In [0]:
deeplabcut.create_training_dataset(config_path)

# Train network
In TPU system, 8 cores are available. So batch size is 8.

In original default setup, max iterations are 1030000. 
If batch size is 8, a iteration processes 8 samples, so 1030000 iters are too much than original. So set batch size is 8 and max iterations are divided by 4 if TPU flag is ON.

And as TPU sysytem limtation, processed image size is fixed(default 400x400).

In this default setup, TPU system trains the network in 3 hours.

In [0]:
deeplabcut.train_network(config_path)

# Tensorboard
If you watch the training process, open the tensorborad.
Open "Google Cloud Shell" in your Storage project and run this command(change the path for your own).

``` bash
$ tensorboard --port=8080 --logdir gs://test_20190814/colab-test-2019- 08-13/dlc-models/iteration-0/colabAug13-trainset95shuffle1/train
```



# Test network


In [0]:
deeplabcut.evaluate_network(config_path, plotting=True)

# Analyze videos
In TPU system, all data for prediction are stored in Google Cloug Storage.
So before next command, you need to run conversion command in local system.

like this,
``` bash
$ deeplabcut.convert_analyze_videos(config_path, [‘/home/user/video/’], ‘gs://test_20190814/video/’, videotype=’mp4’)
```

After upload, run analyze_videos with videotype='tfrecord'.


In [0]:
deeplabcut.analyze_videos(config_path, ['gs://test_20190814/video/'], videotype='tfrecord', save_as_csv=True, destfolder='/content/drive/My Drive/some_dir/result/resnet50-iter0/')